In [0]:
from pyspark.sql.functions import *
from pyspark.sql import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T
import json

#Common python library
import numpy as np
import pandas as pd
import datetime as dt
import unittest

#Shapely : for geometry manipulation
from shapely import geometry
from shapely.geometry import Polygon
import shapely
import shapely.wkt

#Library and dependency for Apache Sedona which gonna help up making spatial join like PostGreSQL with distributed data
#import sedona
from pyspark.sql import SparkSession
#from sedona.utils import SedonaKryoRegistrator, KryoSerializer
#from sedona.register import SedonaRegistrator
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, ArrayType


In [0]:
dbutils.library.installPyPI("libify")
!pip install cryptography
import libify


dbutils.library APIs are deprecated and will be removed in a future DBR release. You can use %pip commands to install notebook-scoped python libraries.
Collecting cryptography
 Downloading cryptography-38.0.1-cp36-abi3-manylinux_2_28_x86_64.whl (4.2 MB)
 | 10 kB 18.6 MB/s eta 0:00:01 |▏ | 20 kB 5.9 MB/s eta 0:00:01 |▎ | 30 kB 8.5 MB/s eta 0:00:01 |▎ | 40 kB 4.5 MB/s eta 0:00:01 |▍ | 51 kB 4.8 MB/s eta 0:00:01 |▌ | 61 kB 5.7 MB/s eta 0:00:01 |▌ | 71 kB 5.7 MB/s eta 0:00:01 |▋ | 81 kB 6.4 MB/s eta 0:00:01 |▊ | 92 kB 6.1 MB/s eta 0:00:01 |▉ | 102 kB 5.3 MB/s eta 0:00:01 |▉ | 112 kB 5.3 MB/s eta 0:00:01 |█ | 122 kB 5.3 MB/s eta 0:00:01 |█ | 133 kB 5.3 MB/s eta 0:00:01 |█ | 143 kB 5.3 MB/s eta 0:00:01 |█▏ | 153 kB 5.3 MB/s eta 0:00:01 |█▎ | 163 kB 5.3 MB/s eta 0:00:01 |█▍ | 174 kB 5.3 MB/s eta 0:00:01 |█▍ | 184 kB 5.3 MB/s eta 0:00:01 |█▌ | 194 kB 5.3 MB/s eta 0:00:01 |█▋ | 204 kB 5.3 MB/s eta 0:00:01 |█▋ | 215 kB 5.3 MB/s eta 0:00:01 |█▊ | 225 kB 5.3 MB/s eta 0:00:01 |█▉ | 235 kB 5.3 MB/s eta 0:00:01 |██ | 245 kB 5.3 MB/s eta 0:00:01 |██ | 256 kB 5.3 MB/s eta 0:00:01 |██ | 266 kB 5.3 MB/s eta 0:00:01 |██▏ | 276 kB 5.3 MB/s eta 0:00:01 |██▏ | 286 kB 5.3 MB/s eta 0:00:01 |██▎ | 296 kB 5.3 MB/s eta 0:00:01 |██▍ | 307 kB 5.3 MB/s eta 0:00:01 |██▍ | 317 kB 5.3 MB/s eta 0:00:01 |██▌ | 327 kB 5.3 MB/s eta 0:00:01 |██▋ | 337 kB 5.3 MB/s eta 0:00:01 |██▊ | 348 kB 5.3 MB/s eta 0:00:01 |██▊ | 358 kB 5.3 MB/s eta 0:00:01 |██▉ | 368 kB 5.3 MB/s eta 0:00:01 |███ | 378 kB 5.3 MB/s eta 0:00:01 |███ | 389 kB 5.3 MB/s eta 0:00:01 |███ | 399 kB 5.3 MB/s eta 0:00:01 |███▏ | 409 kB 5.3 MB/s eta 0:00:01 |███▎ | 419 kB 5.3 MB/s eta 0:00:01 |███▎ | 430 kB 5.3 MB/s eta 0:00:01 |███▍ | 440 kB 5.3 MB/s eta 0:00:01 |███▌ | 450 kB 5.3 MB/s eta 0:00:01 |███▌ | 460 kB 5.3 MB/s eta 0:00:01 |███▋ | 471 kB 5.3 MB/s eta 0:00:01 |███▊ | 481 kB 5.3 MB/s eta 0:00:01 |███▉ | 491 kB 5.3 MB/s eta 0:00:01 |███▉ | 501 kB 5.3 MB/s eta 0:00:01 |████ | 512 kB 5.3 MB/s eta 0:00:01 |████ | 522 kB 5.3 MB/s eta 0:00:01 |████ | 532 kB 5.3 MB/s eta 0:00:01 |████▏ | 542 kB 5.3 MB/s eta 0:00:01 |████▎ | 552 kB 5.3 MB/s eta 0:00:01 |████▎ | 563 kB 5.3 MB/s eta 0:00:01 |████▍ | 573 kB 5.3 MB/s eta 0:00:01 |████▌ | 583 kB 5.3 MB/s eta 0:00:01 |████▋ | 593 kB 5.3 MB/s eta 0:00:01 |████▋ | 604 kB 5.3 MB/s eta 0:00:01 |████▊ | 614 kB 5.3 MB/s eta 0:00:01 |████▉ | 624 kB 5.3 MB/s eta 0:00:01 |████▉ | 634 kB 5.3 MB/s eta 0:00:01 |█████ | 645 kB 5.3 MB/s eta 0:00:01 |█████ | 655 kB 5.3 MB/s eta 0:00:01 |█████▏ | 665 kB 5.3 MB/s eta 0:00:01 |█████▏ | 675 kB 5.3 MB/s eta 0:00:01 |█████▎ | 686 kB 5.3 MB/s eta 0:00:01 |█████▍ | 696 kB 5.3 MB/s eta 0:00:01 |█████▍ | 706 kB 5.3 MB/s eta 0:00:01 |█████▌ | 716 kB 5.3 MB/s eta 0:00:01 |█████▋ | 727 kB 5.3 MB/s eta 0:00:01 |█████▊ | 737 kB 5.3 MB/s eta 0:00:01 |█████▊ | 747 kB 5.3 MB/s eta 0:00:01 |█████▉ | 757 kB 5.3 MB/s eta 0:00:01 |██████ | 768 kB 5.3 MB/s eta 0:00:01 |██████ | 778 kB 5.3 MB/s eta 0:00:01 |██████ | 788 kB 5.3 MB/s eta 0:00:01 |██████▏ | 798 kB 5.3 MB/s eta 0:00:01 |██████▏ | 808 kB 5.3 MB/s eta 0:00:01 |██████▎ | 819 kB 5.3 MB/s eta 0:00:01 |██████▍ | 829 kB 5.3 MB/s eta 0:00:01 |██████▌ | 839 kB 5.3 MB/s eta 0:00:01 |██████▌ | 849 kB 5.3 MB/s eta 0:00:01 |██████▋ | 860 kB 5.3 MB/s eta 0:00:01 |██████▊ | 870 kB 5.3 MB/s eta 0:00:01 |██████▊ | 880 kB 5.3 MB/s eta 0:00:01 |██████▉ | 890 kB 5.3 MB/s eta 0:00:01 |███████ | 901 kB 5.3 MB/s eta 0:00:01 |███████ | 911 kB 5.3 MB/s eta 0:00:01 |███████ | 921 kB 5.3 MB/s eta 0:00:01 |███████▏ | 931 kB 5.3 MB/s eta 0:00:01 |███████▎ | 942 kB 5.3 MB/s eta 0:00:01 |███████▎ | 952 kB 5.3 MB/s eta 0:00:01 |███████▍ | 962 kB 5.3 MB/s eta 0:00:01 |███████▌ | 972 kB 5.3 MB/s eta 0:00:01 |███████▋ | 983 kB 5.3 MB/s eta 0:00:01 |███████▋ | 993 kB 5.3 MB/s eta 0:00:01 |███████▊ | 1.0 MB 5.3 MB/s eta 0:00:01 |███████▉ | 1.0 MB 5.3 MB/s eta 0:00:01 |███████▉ | 1.0 MB 5.3 MB/s eta 0:00:01 |████████ | 1.0 

In [0]:
apr_data_func = libify.importer(globals(),"/Dev/APR_DATA_FUNCTIONS")

dbutils.library APIs are deprecated and will be removed in a future DBR release. You can use %pip commands to install notebook-scoped python libraries.
PyPI package libify has been installed already. The previously installed package is `libify`. To resolve this issue, detach and re-attach the notebook to create a new environment or rename the package.

In [0]:
def node_selection_f(node_list):
  """This function going to select the second node from all node (The most common one)"""  
  if len(node_list) < 2:
    x= node_list[0]
  else:
    x= node_list[1]
  return x

#The UDF to apply it on a pyspark dataframe
node_selection = F.udf(node_selection_f, T.LongType())
def parse_embedding_from_string(x):
    res = json.loads(x)
    return res

#LongType no Integer type.... 
retrieve_embedding = F.udf(parse_embedding_from_string, T.ArrayType(T.LongType()))

In [0]:
#récuperer les données pour le mois février 2020
year = "2020"
month = "02"
df = apr_data_func.get_releves_intermediares(year,month)

In [0]:
view_result = spark.read.csv("/mnt/datalake/tmp/guillaume/temporary/full_eclaireur/result_join_poly",inferSchema=True,header=True)

In [0]:
# on récupere seulement les nodes que sont definis 
landuse = view_result.filter(~F.col("inside")=="false")

In [0]:
#allnode est de type String donc on le change en array à l'aide de la  fonction (retrieve_embedding)
df = df.withColumn("allnodes",retrieve_embedding(F.col("allnodes")))


In [0]:
#récuperer la node ID à partir la colonne allnode(la deuxiéme node)
df = df.withColumn('node_id',node_selection(F.col("allnodes")))

In [0]:
#on fait le jointure suivant la colonne node_ID(node_id pour df et ID pour landuse)
df_join = df.join(landuse, df.node_id == landuse.ID, how = "left")

In [0]:
#drop les colonnes de plus , remplace null par inconnue, recupere seulement les colonnes qu'on a besoin
df_join = df_join.drop("ID").drop("inside").na.fill("inconnue")
df_join = df_join.select("rideId","dateentry","node_id","landuse","deviceid")

In [0]:
#utiliser window pour afficher date min et max pour chaque rideid suivant les differents landuse , ordonner par la date 
windows = Window.partitionBy("rideId","landuse").orderBy(F.col("dateentry").asc())
df_groupby = (df_join.withColumn('datemax', max('dateentry').over(windows))
 .withColumn('datemin', min('dateentry').over(windows)))



In [0]:
#afficher pour chaque rideid : le temps de debut et fin ,et avec les differents landuse 
data_final1=df_groupby.groupby("rideId","datemin").agg(F.max("datemax").alias("end_time"),
                                          (F.first("landuse").alias("landuse")))

In [0]:
data_final1.count()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2051775506245361> in <module> 
 ----> 1 data_final1 . count ( ) 

 NameError : name 'data_final1' is not defined

In [0]:
#renommer les colonnes 
data_final=data_final1.withColumnRenamed('datemin','start_time')

In [0]:
data_final.write.option('header',True).mode('overwrite').csv('/mnt/datalake/tmp/amani/landuse_process/02_2020_ride')